In [1]:
using Pkg
Pkg.add("Latexify");
Pkg.add("LaTeXStrings");
Pkg.add("LinearAlgebra");
Pkg.add("JuMP");
Pkg.add("Clp");
Pkg.add("PyPlot");
Pkg.add("NamedArrays")
Pkg.add("CSV");
Pkg.add("DataFrames");
Pkg.add("Ipopt");

   Updating registry at `~/.julia/registries/General`
######################################################################### 100.0%
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.t

# CS/ECE/ISyE 524 - Spr 2021 - HW 9
### Jacob Lorenz

# 1. Checked luggage.

You're trying to pack as many souvenirs as possible to bring home from your trip, but your suitcase has a limited capacity. It can hold a maximum of 30 pounds of weight and 15 gallons of volume. Which souvenirs should you pack? The weights and volumes are as follows:

|Souvenir number |1 |2 |3 |4 |5 |6 |7 |8 |9 |10|
|:---------------|--|--|--|--|--|--|--|--|--|--|
|Weight (lbs)    |5 |6 |7 |6 |4 |6 |7 |3 |8 |5 |
|Volume (gal)    |2 |4 |5 |3 |3 |2 |3 |1 |2 |4 |

In [10]:
# problem data (we'll use an array here)
weights = [ 5, 6, 7, 6, 4, 6, 7, 3, 8, 5 ]
volumes = [ 2, 4, 5, 3, 3, 2, 3, 1, 2, 4 ]
W = 30
V = 15
n = length(volumes) # total number of souvenirs

# Template used from Knapsack.ipynb
using JuMP, Gurobi
m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m, "OutputFlag", 0)

@variable(m, z[1:n], Bin )
@constraint(m, sum( weights[i]*z[i] for i=1:n) <= W )
@constraint(m, sum( volumes[i]*z[i] for i=1:n) <= V )
@objective(m, Max, sum( z[i] for i=1:n) )

optimize!(m)

println()
println("Binary Solution Vector: ")
println(value.(z))
println()
print("Total Number of Souvenirs: ")
println(objective_value(m))
println()
print("Total Weight: ")
println(sum(value.(z).*weights))
print("Total Volume: ")
println(sum(value.(z).*volumes))

Academic license - for non-commercial use only - expires 2021-05-01

Binary Solution Vector: 
[1.0, -0.0, -0.0, 1.0, 1.0, 1.0, 0.0, 1.0, -0.0, 1.0]

Total Number of Souvenirs: 6.0

Total Weight: 29.0
Total Volume: 15.0


In conclusion of problem 1, we see that the optimal collection of souvenirs to keep in order to maximize the total number of souvenirs irresepective of any value is:
- Object 1
- Object 4
- Object 5
- Object 6
- Object 8
- Object 10

I am confident in this collection since the total volume has a strict equality with the volume limit, and the total weight is only 1 unit off the weight limit. Hope we left some room to bring our clothes back home.

# 2. Comquat Computers

Comquat owns four production plants at which personal computers are produced. Comquat can sell up to 20,000 computers per year at a price of \$3,500 per computer. For each plant the production capacity, cost per computer, and fixed cost of operating the plant for a year are given below. Determine how Comquat can maximize its yearly profit from computer production.

| Plant | Production capacity | Plant fixed cost (\$ Million) | Cost per computer (\$) |
|:-----:|:-------------------:|:-----------------------------:|:----------------------:|
| 1     | 10,000              |   9                           |  1,000                 |
| 2     |  8,000              |   5                           |  1,700                 |
| 3     |  9,000              |   3                           |  2,300                 |
| 4     |  6,000              |   1                           |  2,900                 |

In [106]:
using JuMP, Gurobi
m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m, "OutputFlag", 0)

price     = 3.5e3                       # price at which we can sell a single computer 
                                        # (regardless of where it is produced)
cap       = 20000                       # maximum computers that can be sold per year

capacity  = [10000, 8000, 9000, 6000]   # production capacity for each plant
fixedcost = [9e6, 5e6, 3e6, 1e6]        # fixed cost for each plant
itemcost  = [1e3, 1.7e3, 2.3e3, 2.9e3]  # cost per computer for each factory
n = length(capacity)

@variable(m, z[1:n], Bin)
@variable(m, x[1:n] >= 0)
@constraint(m, sum(x[i] for i in 1:n) <= cap)
for i in 1:n
   @constraint(m, x[i] <= capacity[i]) 
end
@expression(m, FixedCost, sum(z[i]*fixedcost[i] for i in 1:n))
@expression(m, MaterialCost, sum(x[i]*itemcost[i] for i in 1:n))
@expression(m, Revenue, price*sum(x[i]*z[i] for i in 1:n))

@objective(m, Max, Revenue - FixedCost - MaterialCost)

optimize!(m)
println()
println("The maximal profit is: ", objective_value(m))
println()
println("Here is a tally of factory number, whether it's open or not, and how many computers it should produce.")
value.([1:4 z x])

Academic license - for non-commercial use only - expires 2021-05-01

The maximal profit is: 2.56e7

Here is a tally of factory number, whether it's open or not, and how many computers it should produce.


4×3 Array{Float64,2}:
 1.0  1.0  10000.0
 2.0  1.0   8000.0
 3.0  0.0      0.0
 4.0  1.0   2000.0

In conclusion of problem 2, we see that the optimal strategy is to:
- Open plant 1 and produce 10,000 computers
- Open plant 2 and produce 8,000 computers
- Close plant 3
- Open plant 4 and produce 2,000 computers

The total profit from this strategy is $\$$25.6 Million

An interesting observation we can see from these results is that the third factory is the least cost efficient, and the fourth factory is the second least cost efficient. We can tell this because the first two plants max out their capacity, and the remaining capacity is filled in plant three. The fourth factory was factored in because the production of 2,000 computers results in revenue of $\$$7 Million, the startup cost is only $\$$1 Million, and the material cost for 2,000 computers is $\$$5.8 Million, resulting in a profit of $\$$200 Thousand for the production at that company. An interesting hypothetical to notice is that if material costs increased by $\gt \$$100 per computer at plant 4 or if the total production cap across all plants decreased by 334 computers, it would no longer be in the best financial interest of the company to produce computers at plant 4.

# 3. ABC Investments

ABC Inc. is considering several investment options.  Each option has a minimum investment required as well as a maximum investment allowed. These restrictions, along with the expected return are summarized in the following table (figures are in millions of dollars):

|Option | Minimum investment | Maximum investment | Expected return (%)  |
|:-----:|:------------------:|:------------------:|:--------------------:|
|  1    |      3             |      27            |     13               |
|  2    |      2             |      12            |      9               |
|  3    |      9             |      35            |     17               |
|  4    |      5             |      15            |     10               |
|  5    |     12             |      46            |     22               |
|  6    |      4             |      18            |     12               |

Because of the high-risk nature of Option 5, company policy requires that the total amount invested in Option 5 be no more that the combined amount invested in Options 2, 4 and 6.  In addition, if an investment is made in Option 3, it is required that at least a minimum investment be made in Option 6.  ABC has \$80 million to invest and obviously wants to maximize its total expected return on investment.  Which options should ABC invest in, and how much should be invested?

In [107]:
using JuMP, Gurobi
m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m, "OutputFlag", 0)

maxInvestment = 80
lows  = [  3,  2,  9,  5, 12,  4 ]  # minimum investment
highs = [ 27, 12, 35, 15, 46, 18 ]  # maximum investment
ret   = [ 13,  9, 17, 10, 22, 12 ]  # expected return
n = length(lows)

@variable(m, z[1:n], Bin)
@variable(m, x[1:n])

for i in 1:n
    # Set the constraints for maximum and minimum investments for an option if we choose to invest at all
    @constraint(m, x[i] <= z[i]*highs[i]) 
    @constraint(m, x[i] >= z[i]*lows[i])
end

@constraint(m, x[6] >= z[3]*lows[6]) # Custom constraint for investment in option 3
@constraint(m, x[5] <= x[2] + x[4] + x[6]) # Custom constraint for investment in option. 5
@constraint(m, sum(x[i] for i in 1:n) <= maxInvestment) # Total investment canot exceed investment cap

# Net Profit = Sum (Principle * (1 + return/100)) - Sum (Principle)
@expression(m, TotalReturn, sum(x[i]*(1+(ret[i]/100)) for i in 1:n) - maxInvestment)

@objective(m, Max, TotalReturn)

optimize!(m)

println("The maximum return on investment (net profit) is: \$", objective_value(m), " million.")
println("This is an average return of: ", 100*objective_value(m)/value(sum(x)), " %")
println("Here is a tally of each investment, whether we invest or not, and how much we invest (in millions of \$).")
value.([1:6 z x])

Academic license - for non-commercial use only - expires 2021-05-01
The maximum return on investment (net profit) is: $13.5 million.
This is an average return of: 16.875 %
Here is a tally of each investment, whether we invest or not, and how much we invest (in millions of $).


6×3 Array{Float64,2}:
 1.0  0.0   0.0
 2.0  0.0   0.0
 3.0  1.0  34.0
 4.0  1.0   5.0
 5.0  1.0  23.0
 6.0  1.0  18.0

From the results shown above, we can see that the optimal investment startegy is:
- $\$$0 in Option 1
- $\$$0 in Option 2
- $\$$34 Million in Option 3
- $\$$5 Million in Option 4
- $\$$23 Million in Option 5
- $\$$18 Million in Option 6

In hindsight, this result makes sense. Option 5 has the highest return at 22\%, so it seems natural to want to put as much as possible into it; however, it comes with the downside that you need an equivalent investment total amongst Options 2, 4, and 6, which have a lower expected return than Option 3. Given that Option 2 is the worst return, the model does not even consider it. Additionally, Option 6 has the highest return among 2, 4, and 6, so it makes sense that we use the maximum investment in Option 6 and then fill the remainder of the equality requirement with Option 4.

All of this is then traded off with the second highest returning Option, Option 3 at 17\%. The expected return of Option 3 is the average of returns between Option 5 and Option 6; however, it is higher than the average return between Option 5 and Option 4. Therefore, to offset the loss taken on investing in Option 4 due to the requirement for it having a minimum due to investing in Option 5, the allocation amount to Option 3 is the largest. If the maximum investment in Option 6 increased by $\$$1 Million, we would see no loss from removing $\$$2 Million from Option 3 and allocating an additional $\$$1 Million to each Option 5 and Option 6. If the maximum investment in Option 6 decreased by $\$$1 Million; however, we would see a larger relative allocation to Option 3 instead of a higher allocation split between Option 4 and Option 5.

# 4. Paint production

As part of its weekly production, a paint company produces five batches of paints, always the same, for some big clients who have a stable demand.  Every paint batch is produced in a single production process, all in the same blender that needs to be cleaned between each batch.  The durations of blending paint batches 1 to 5 are 40, 35, 45, 32 and 50 minutes respectively.  The cleaning times depend of the colors and the paint types.  For example, a long cleaning period is required if an oil-based paint is produced after a water-based paint, or to produce white paint after a dark color.  The times are given in minutes in the following matrix $A$ where $A_{ij}$ denotes the cleaning time after batch $i$ if it is followed by batch $j$.

$$
A = \begin{bmatrix}
 0&11& 7&13&11 \\
 5& 0&13&15&15 \\
13&15& 0&23&11 \\
 9&13& 5& 0& 3 \\
 3& 7& 7& 7& 0
\end{bmatrix}
$$

Since the company has other activities, it wishes to deal with this weekly production in the shortest possible time (blending and cleaning).  What is the corresponding order of paint batches?  The order will be applied every week, so the cleaning time between the last batch of one week and the first of the following week needs to be accounted for in the total duration of cleaning.
                

In [110]:
# This is a traveling salesman problem! cities correspond to paint batches, and the time it takes to travel
# from one city to the next is the time it takes to clean the blender in between two particular batches.

# Because the cleaning time between the last batch of one week and the first batch of the following week needs
# to be accounted for, this is EXACTLY a traveling salesman problem!

# A[i,j] is the time it takes to travel i --> j.
A = [  0  11   7  13  11
       5   0  13  15  15
      13  15   0  23  11
       9  13   5   0   3
       3   7   7   7   0 ]

# time it takes to blend a particular batch. This information isn't important, because we need to blend each batch
# every week no matter what order they're blended in. So this is a fixed cost!
b = [ 40, 35, 45, 32, 50 ]

using JuMP, Gurobi
m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m, "OutputFlag", 0)

n = length(b)  # number of paints

@variable(m, x[1:n,1:n], Bin)

for i in 1:n
    @constraint(m, x[i,i] == 0) # Cannot have 2 subsequent batches of the same paint
    @constraint(m, sum(x[i,1:n]) == 1) # Cannot have more than one subsequent batch 
    @constraint(m, sum(x[1:n,i]) == 1) # Cannot have more than one batch going at the same time
end

# Prevent loops
for i in 1:n
    for j in 1:n
       @constraint(m, x[i,j]+x[j,i] <= 1) 
    end
end

@expression(m, TotalCleaningTime, sum(x[i,j]*A[i,j] for i in 1:n, j in 1:n))

@objective(m, Min, TotalCleaningTime)

optimize!(m)

println()
println("Optimal ordering is:")
X = value.(x);

BatchOrder = zeros(n+1)
BatchOrder[1] = 1; # Start with batch 1, this is arbitrary
continueLooping = true
currentBatch = 1;
currentPosition = 2
while(continueLooping)
    batchSuccessor = 0;
    for j = 1:n
       if (X[currentBatch,j] == 1)
           batchSuccessor = j 
        end
    end
    
    BatchOrder[currentPosition] = batchSuccessor;
    currentBatch = batchSuccessor;
    currentPosition += 1
    if (currentPosition > n+1)
       continueLooping = false 
    end
end

print(Integer(BatchOrder[1]))
for i = 2:n+1
    print(" --> ")
    print(Integer(BatchOrder[i]))
end
println()
println()

println("Minimum cleanup time is: ", objective_value(m), " min, (plus ", sum(b), " min of blending)")

Academic license - for non-commercial use only - expires 2021-05-01

Optimal ordering is:
1 --> 4 --> 3 --> 5 --> 2 --> 1

Minimum cleanup time is: 41.0 min, (plus 202 min of blending)


From the result shown above, we see that the optimal ordering of batch production is Batch 1, Batch 4, Batch 3, Batch 5, Batch 2.

The thing is, this ordering is relative, so we could realistically start with any Batch number, and as long as we follow this sequence of successors for the remainder of the week's production cycle and start with the original Batch number for the first production in the following week, we will still achieve the same optimal minimum production time. 

The minimum production time was found to be a total of 243 minutes, where 202 minutes of that time are unavoidable no matter the order since it is the time it takes to blend all of the paints. 41 minutes of that time are where the optimization comes in, as that is the lowest possible total cleaning time between batches with the requirement to blend every batch at least once.

Interesting enough, I found the Maximum cleaning time and sequence just out of curiosity, and it came to be 65 minutes of cleaning time, and the sequence was the exact opposite of the Minimization sequence (1,4,3,5,2,1). I suppose the people who created this problem wanted an easy way of converting the answer if they decide to make it a Maximization problem versus a Minimization problem ;), what a cleverly-crafted cost matrix.